In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import xesmf as xe

In [2]:
study_timeperiod = pd.date_range(start="2014-09", end="2017-04", freq="1M")
print(f"Start: {study_timeperiod[0]}, End: {study_timeperiod[-1]}")

Start: 2014-09-30 00:00:00, End: 2017-03-31 00:00:00


# Construct inputs datasets

## MOZART

In [3]:
file_paths = []
for month in study_timeperiod:
    yyyy, mm, _ = str(month).split("-")
    file_paths.append(
        f"../1_transport/intermediates/MOZART/output/BasisFnsUpdated/{yyyy}{mm}/"
        f"BasisFnsUpdated.mz4.h0.{yyyy}-{mm}-01-03600.nc"
    )


In [4]:
def prepare_mozart_input(ds):
    ds["time"] = pd.to_datetime(ds.date.values, format="%Y%m%d") + pd.to_timedelta(
        ds.datesec.values, unit="seconds"
    )
    # extract co2 emissions array
    return ds["CO2_SRF_EMIS_avrg"]


# concatenate CO2_SRF_EMIS_avrg variable from MOZART diagnostics
with xr.open_mfdataset(file_paths, preprocess=prepare_mozart_input, decode_times=False) as ds:
    da_mozart = ds["CO2_SRF_EMIS_avrg"]

# one observation for every hour of every day in study period, in units kg/m^2/s
da_mozart


<xarray.DataArray 'CO2_SRF_EMIS_avrg' (time: 22632, lat: 96, lon: 144)>
dask.array<concatenate, shape=(22632, 96, 144), dtype=float32, chunksize=(744, 96, 144), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 -90.0 -88.11 -86.21 -84.32 ... 84.32 86.21 88.11 90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) datetime64[ns] 2014-09-01T01:00:00 ... 2017-04-01
Attributes:
    units:    KG/M^2/S

In [5]:
# da_mozart.to_netcdf("../data/mozart_inputs.nc")

In [6]:
# define end of study period
mozart_datetime_end = da_mozart.time.values.max()

## GEOS Chem

In [7]:
def extract_co2_emission_geoschem(ds):
    return ds["EmisCO2_Total"]


geos_chem_path_glob = (
    "../1_transport/intermediates/GEOS_Chem/runs/run.v12.3.2.base/output/HEMCO_diagnostics.*.nc"
)
with xr.open_mfdataset(geos_chem_path_glob, preprocess=extract_co2_emission_geoschem) as ds:
    # compute the sum over the pressure levels
    da_geoschem = ds["EmisCO2_Total"].sum(dim=["lev"])

    # clip to available mozart date range
    da_geoschem = da_geoschem.where(da_geoschem["time"] <= mozart_datetime_end, drop=True)

# one observation for every hour of every day in study period, in units kg/m^2/s
da_geoschem


<xarray.DataArray 'EmisCO2_Total' (time: 22632, lat: 91, lon: 144)>
dask.array<where, shape=(22632, 91, 144), dtype=float32, chunksize=(744, 91, 144), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2014-09-01T00:30:00 ... 2017-03-31T23:30:00
  * lon      (lon) float64 -180.0 -177.5 -175.0 -172.5 ... 172.5 175.0 177.5
  * lat      (lat) float64 -89.5 -88.0 -86.0 -84.0 -82.0 ... 84.0 86.0 88.0 89.5

In [8]:
# da_geoschem.to_netcdf("../data/geoschem_inputs.nc")

# Construct outputs datasets

In [9]:
target_grid = xr.Dataset(
    {
        "lat": (["lat"], np.arange(-89.5, 90.5, 1.0)),
        "lon": (["lon"], np.arange(-180, 180, 1.0)),
    }
)
target_grid

<xarray.Dataset>
Dimensions:  (lat: 180, lon: 360)
Coordinates:
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 -180.0 -179.0 -178.0 -177.0 ... 177.0 178.0 179.0
Data variables:
    *empty*

## MOZART

In [10]:
with xr.open_mfdataset(file_paths, decode_times=False) as ds:
    # setup times correctly
    ds["time"] = pd.to_datetime(ds["date"].values, format="%Y%m%d") + pd.to_timedelta(
        ds["datesec"].values, unit="seconds"
    )
    ds_mozart = ds[["CO2_VMR_avrg", "P0", "PS", "hyai", "hybi"]]
    # compute pressure edge
    ds_mozart["pressure_edge"] = (
        ds_mozart["P0"] * ds_mozart["hyai"] + ds_mozart["PS"] * ds_mozart["hybi"]
    ) / 100
     # shift longitude coordinate reference
    ds_mozart["lon"] = ds_mozart["lon"] - 180.0
    # regrid to 1x1 degree
    ds_mozart = xe.Regridder(ds_mozart, target_grid, "bilinear")(ds_mozart)

# one observation for every hour of every day in study period, in units VMR, regridded
ds_mozart


<xarray.Dataset>
Dimensions:        (time: 22632, lev: 56, lat: 180, lon: 360, ilev: 57)
Coordinates:
  * ilev           (ilev) float32 1.651 2.085 2.62 3.276 ... 970.0 985.0 1e+03
  * lev            (lev) float32 1.868 2.353 2.948 3.677 ... 962.5 977.5 992.5
  * time           (time) datetime64[ns] 2014-09-01T01:00:00 ... 2017-04-01
  * lon            (lon) float64 -180.0 -179.0 -178.0 ... 177.0 178.0 179.0
  * lat            (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
Data variables:
    CO2_VMR_avrg   (time, lev, lat, lon) float32 dask.array<chunksize=(720, 56, 180, 360), meta=np.ndarray>
    PS             (time, lat, lon) float32 dask.array<chunksize=(720, 180, 360), meta=np.ndarray>
    pressure_edge  (time, ilev, lat, lon) float32 dask.array<chunksize=(720, 57, 180, 360), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

## GEOS Chem

In [11]:
geoschem_levedge_glob = (
    "../1_transport/intermediates/GEOS_Chem/runs/run.v12.3.2.base/output/"
    "GEOSChem.LevelEdgeDiags.*_0000z.nc4"
)
with xr.open_mfdataset(geoschem_levedge_glob) as ds:
    da_pressure_level = ds["Met_PEDGE"]

geoschem_spec_glob = (
    "../1_transport/intermediates/GEOS_Chem/runs/run.v12.3.2.base/output/"
    "GEOSChem.SpeciesConc.*_0000z.nc4"
)
with xr.open_mfdataset(geoschem_spec_glob) as ds:
    ds_geoschem = ds[["SpeciesConc_CO2", "AREA"]]
    ds_geoschem["Met_PEDGE"] = da_pressure_level
    # clip to available mozart date range
    ds_geoschem = ds_geoschem.where(ds_geoschem["time"] <= mozart_datetime_end, drop=True)
    # regrid to 1x1 degree
    ds_geoschem = xe.Regridder(ds_geoschem, target_grid, "bilinear")(ds_geoschem)

# one observation for every hour of every day in study period, in units mole per mole dry, regridded
ds_geoschem


<xarray.Dataset>
Dimensions:          (time: 22632, lev: 47, lat: 180, lon: 360, ilev: 48)
Coordinates:
  * time             (time) datetime64[ns] 2014-09-01T00:30:00 ... 2017-03-31...
  * lev              (lev) float64 0.9925 0.9775 0.9625 ... 0.0001387 3.8e-05
  * ilev             (ilev) float64 1.0 0.985 0.97 ... 0.0002113 6.6e-05 1e-05
  * lon              (lon) float64 -180.0 -179.0 -178.0 ... 177.0 178.0 179.0
  * lat              (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
Data variables:
    SpeciesConc_CO2  (time, lev, lat, lon) float32 dask.array<chunksize=(24, 47, 180, 360), meta=np.ndarray>
    AREA             (time, lat, lon) float32 dask.array<chunksize=(24, 180, 360), meta=np.ndarray>
    Met_PEDGE        (time, ilev, lat, lon) float32 dask.array<chunksize=(24, 48, 180, 360), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear